In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from pyearth import Earth
import shap
import os
import warnings


warnings.filterwarnings("ignore")
os.makedirs("results", exist_ok=True)

In [23]:
X = pd.read_csv("./data/x_train.txt", sep=" ", header=None)
y = pd.read_csv("./data/y_train.txt", header=None).iloc[:, 0].to_numpy()
X.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,20.454647,16.739345,39.811892,24.955468,27.088535,17.116793,25.166957,23.364508,17.223886,18.339175,...,13.235314,5.160379,29.194846,17.298314,6.414267,7.780568,6.840910,18.295197,10.014028,6.938318
1,16.175225,10.483281,27.471017,18.509824,19.045353,15.039082,21.354915,15.790575,13.912508,13.772518,...,13.355832,2.609716,8.624576,9.371632,11.789219,9.205471,15.204468,8.358906,8.529152,8.021473
2,10.577212,10.795115,24.621388,17.264747,14.221610,8.754692,18.399259,11.358798,15.432650,14.842153,...,15.179359,10.200144,12.645303,12.147416,8.899863,13.954543,12.356942,16.364696,3.817956,4.094035
3,26.299206,13.471215,51.725934,40.786947,26.052414,33.200702,51.014330,27.685009,33.107991,25.359457,...,7.693654,6.359187,5.760296,5.699580,9.895795,17.011648,12.031000,14.637973,10.172737,10.525373
4,23.193955,20.037969,37.780290,28.983748,25.510508,15.970348,27.930757,20.707354,25.341768,27.118987,...,7.068407,7.180632,12.517752,9.026493,7.748172,10.363749,7.099588,12.467672,11.545619,9.098600


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(eval_metric='logloss')
}

for name, model in models.items():
    score = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    print(f"{name}: ACC = {score.mean():.4f} ± {score.std():.4f}")
    


Logistic Regression: ACC = 0.6158 ± 0.0175
Random Forest: ACC = 0.7013 ± 0.0051
SVM: ACC = 0.6590 ± 0.0164
XGBoost: ACC = 0.6998 ± 0.0130


In [8]:
from sklearn.linear_model import LogisticRegression

param_dist_lr = {
    "C": [0.01, 0.1],    # Inverse of regularization strength
    "penalty": ["l1", "l2", "elasticnet"],
    "solver": ["saga"],
    "l1_ratio": [0, 0.5, 1]          # Only used if penalty='elasticnet'
}

rs_lr = RandomizedSearchCV(
    LogisticRegression(max_iter=5000),
    param_distributions=param_dist_lr,
    scoring='accuracy',
    cv=5,
    n_iter=20,
    random_state=42,
    n_jobs=-1
)

rs_lr.fit(X_train_scaled, y_train)
print("Best Logistic Regression Params:", rs_lr.best_params_)


Best Logistic Regression Params: {'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.5, 'C': 0.01}


In [27]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [5],
    'learning_rate': [0.01, 0.001],
    'subsample': [0.8],
}

rs_xg = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=25,
    scoring='accuracy',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rs_xg.fit(X_train, y_train)
print("Best XGBoost Params:", rs_xg.best_params_)

Best XGBoost Params: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.001}


In [9]:
from sklearn.ensemble import RandomForestClassifier

param_dist_rf = {
    "n_estimators": [100, 300],
    "max_depth": [5, 10]
}

rs_rf = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist_rf,
    scoring='accuracy',
    cv=5,
    n_iter=20,
    random_state=42,
    n_jobs=-1
)

rs_rf.fit(X_train_scaled, y_train)
print("Best Random Forest Params:", rs_rf.best_params_)


Best Random Forest Params: {'n_estimators': 300, 'max_depth': 10}


In [10]:
from sklearn.svm import SVC

param_dist_svm = {
    "C": [0.1, 1, 10, 100],
    "gamma": ["scale", "auto"]
}

rs_svm = RandomizedSearchCV(
    SVC(probability=True),
    param_distributions=param_dist_svm,
    scoring='accuracy',
    cv=5,
    n_iter=20,
    random_state=42,
    n_jobs=-1
)

rs_svm.fit(X_train_scaled, y_train)
print("Best SVM Params:", rs_svm.best_params_)


Best SVM Params: {'gamma': 'scale', 'C': 1}


In [16]:
from sklearn.metrics import classification_report

best_model = rs_lr.best_estimator_
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print("ACC:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


ACC: 0.693
              precision    recall  f1-score   support

           0       0.69      0.74      0.71       512
           1       0.70      0.64      0.67       488

    accuracy                           0.69      1000
   macro avg       0.69      0.69      0.69      1000
weighted avg       0.69      0.69      0.69      1000



In [19]:
from sklearn.metrics import classification_report

best_model = rs_rf.best_estimator_
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print("ACC:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

ACC: 0.69
              precision    recall  f1-score   support

           0       0.70      0.70      0.70       512
           1       0.68      0.68      0.68       488

    accuracy                           0.69      1000
   macro avg       0.69      0.69      0.69      1000
weighted avg       0.69      0.69      0.69      1000



In [20]:
from sklearn.metrics import classification_report

best_model = rs_svm.best_estimator_
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print("ACC:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

ACC: 0.655
              precision    recall  f1-score   support

           0       0.66      0.68      0.67       512
           1       0.65      0.63      0.64       488

    accuracy                           0.66      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.65      0.66      0.65      1000



In [28]:
from sklearn.metrics import classification_report

best_model = rs_xg.best_estimator_
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print("ACC:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

ACC: 0.512
              precision    recall  f1-score   support

           0       0.51      1.00      0.68       512
           1       0.00      0.00      0.00       488

    accuracy                           0.51      1000
   macro avg       0.26      0.50      0.34      1000
weighted avg       0.26      0.51      0.35      1000

